# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5d00258b50>
├── 'a' --> tensor([[-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827]])
└── 'x' --> <FastTreeValue 0x7f5d002bbe80>
    └── 'c' --> tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                        [ 0.1379, -0.4206, -2.2079, -0.9166],
                        [-0.5703,  0.7308, -0.9355, -1.5028]])

In [4]:
t.a

tensor([[-0.4218,  0.0410,  0.7314],
        [-0.0452,  0.4764, -0.3827]])

In [5]:
%timeit t.a

86 ns ± 1.35 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5d00258b50>
├── 'a' --> tensor([[ 0.8757,  1.0854,  1.8287],
│                   [-0.0504, -1.3000,  0.1499]])
└── 'x' --> <FastTreeValue 0x7f5d002bbe80>
    └── 'c' --> tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                        [ 0.1379, -0.4206, -2.2079, -0.9166],
                        [-0.5703,  0.7308, -0.9355, -1.5028]])

In [7]:
%timeit t.a = new_value

98.6 ns ± 1.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827]]),
    x: Batch(
           c: tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                      [ 0.1379, -0.4206, -2.2079, -0.9166],
                      [-0.5703,  0.7308, -0.9355, -1.5028]]),
       ),
)

In [10]:
b.a

tensor([[-0.4218,  0.0410,  0.7314],
        [-0.0452,  0.4764, -0.3827]])

In [11]:
%timeit b.a

73.7 ns ± 1.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1498, -0.1574,  0.5119],
               [-0.1084,  0.1355, -0.5414]]),
    x: Batch(
           c: tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                      [ 0.1379, -0.4206, -2.2079, -0.9166],
                      [-0.5703,  0.7308, -0.9355, -1.5028]]),
       ),
)

In [13]:
%timeit b.a = new_value

656 ns ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.1 µs ± 17.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.2 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

281 µs ± 7.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

267 µs ± 3.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5d002587c0>
├── 'a' --> tensor([[[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]],
│           
│                   [[-0.4218,  0.0410,  0.7314],
│                    [-0.0452,  0.4764, -0.3827]]])
└── 'x' --> <FastTreeValue 0x7f5d00258340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49.1 µs ± 689 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5c494eafd0>
├── 'a' --> tensor([[-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827],
│                   [-0.4218,  0.0410,  0.7314],
│                   [-0.0452,  0.4764, -0.3827]])
└── 'x' --> <FastTreeValue 0x7f5c494eab20>
    └── 'c' --> tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                        [ 0.1379, -0.4206, -2.2079, -0.9166],
                 

In [23]:
%timeit t_cat(trees)

44.5 µs ± 624 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

85.9 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]],
       
               [[-0.4218,  0.0410,  0.7314],
                [-0.0452,  0.4764, -0.3827]]]),
    x: Batch(
           c: tensor([[[ 0.0537,  0.2184,  0.7362, -0.5102],
                       [ 0.1379, -0.4206, -2.2079, -0.9166],
                       [-0.5703,  0.7308, -0.9355, -1.5028]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827],
               [-0.4218,  0.0410,  0.7314],
               [-0.0452,  0.4764, -0.3827]]),
    x: Batch(
           c: tensor([[ 0.0537,  0.2184,  0.7362, -0.5102],
                      [ 0.1379, -0.4206, -2.2079, -0.9166],
                      [-0.5703,  0.7308, -0.9355, -1.5028],
                      [ 0.0537,  0.2184,  0.7362, -0.5102],
                      [ 0.1379, -

In [28]:
%timeit Batch.cat(batches)

197 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

542 µs ± 9.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
